In [4]:
import numpy as np
import matplotlib.pyplot as plt
import cantera as ct
from CoolProp.CoolProp import PropsSI

# Pull values from Master Sheet

In [18]:
import os
import sys

# Navigate to the parent directory ('TheRocket')
os.chdir('C:/Users/Kidus/Desktop/TheRocket')
# Now import your module
import pythonExcelPull

C:\Users\Kidus\Desktop\TheRocket


In [20]:
import pandas as pd
import pythonExcelPull

#Must have openpyxl installed in python environment, separately from the pandas install! Pandas will call on this
# Read the Excel file
df = pd.read_excel('C:/Users/Kidus/Desktop/TheRocket/MasterParameters.xlsx')

# List of variables to pull from the Excel file
variables_to_pull = ['heightPayload',
                     'heightNoseCone',
                     'heightMainBay',
                     'heightAvBay',
                     'heightDrogueBay',
                     'heightCoupler3',
                     'heightOx',
                     'heightFuel',
                     'heightCoupler2',
                     'heightCoupler1',
                     'heightEngine',
                     'OD',
                     'tankThick'
]

# Creating an object to store excel variables
class Parameters:
    pass
ex = Parameters()

# Storing excel variables in object
for variable in variables_to_pull:
    value = pythonExcelPull.pythonExcelPull(df, variable)
    setattr(ex, variable, value)

# Input Chamber Parameters

In [22]:
To = 3200 # Stagnation temperature (Kelvin)
Po = 400 # Stagnation pressure (PSI)
OF = 10.2 # O/F ratio

# Input Chamber Geometry

In [23]:
# Pull from sizing notebook/spreadsheet

# Input Nitrous & Ethane States

In [34]:
# Nitrous, saturated liquid at 750 PSI

coolant_name = 'NITROUSOXIDE'
P_ni = 5.171e+6 # Pa

rho_ni = PropsSI('D','P', P_ni,'Q',0, coolant_name)
cp_ni = PropsSI('C','P', P_ni,'Q',0, coolant_name)
k_ni = 17.4 # CoolProp does not store this value for nitrous
mu_ni = 3.237e-3 # CoolProp does not store this value for nitrous
Pr_ni = (cp_ni * mu_ni)/k_ni



# Density, Cp, Mu, Pr, Re, Nu, Temperature

# Ethane, saturated liquid at 550 PSI

fuel_name = 'ETHANE'
P_et = 3.792e+6 # Pa

rho_et = PropsSI('D','P', P_et,'Q',0, fuel_name)
cp_et = PropsSI('C','P', P_et,'Q',0, fuel_name)
k_et = PropsSI('L','P', P_et,'Q',0, fuel_name)
mu_et = PropsSI('V','P', P_et,'Q',0, fuel_name)
Pr_et = (cp_et * mu_et)/k_et

# Density, Cp, Mu, Pr, Re, Nu, Temperature

Note: Nitrous properties that weren't listed in CoolProp came from the following sites:
1. Thermal conductivity (k): https://www.engineersedge.com/heat_transfer/thermal-conductivity-gases.htm
2. Viscosity (mu): https://rocketprops.readthedocs.io/en/latest/n2o_prop.html

# Input Cooling Channel Geometry

In [24]:
w = np.linspace(5e-4, 5e-3, 1000) # Channel width in meters
h = np.linspace(5e-4, 5e-3, 1000) # Channel hieght in meters

# Define Heat Analysis Functions

In [35]:
def stat_temp(M, gamma, To):
    return To*(1 + ((gamma - 1)/2) * M**2)**(-1)

def stat_pressure(M, gamma, Po):
    return Po*(1 + ((gamma - 1)/2) * M**2)**(-(gamma)/(gamma - 1))

def T_rec():
    return stat_temp(M, gamma, To)*(1 + ((gamma - 1)/2)*((Pr)**(1/3))*(M**2))

def bartz(): # Bartz equation returns the enthalpy of the combusted gas
    return (0.026/(D*(0.2)))*((cp*((mu)**(0.2)))/(Pr**(0.6)))*((rho*v)**(0.8))*(rho_am/rho)*((mu_am/mu)**(0.2))

def cyl_conv_R(r_o, r_i, k_ni):
    return np.log(r_o/r_i)/(2*np.pi*k)

def cyl_cond_R(r, h):
    return 1/(2*np.pi*r*h)

def R_net():
    return cyl_cond_R(r, h_g) + cyl_cond_R(r, h_c) + np.log(r_o/r_i)/(2*np.pi*k_ni)
    
def heat_flow(T_h, T_c):
    return (T_h - T_c)/R_net

